# kaggle_quora: baseline of yuhaitao

比赛baseline

参考:

https://www.kaggle.com/shujian/single-rnn-with-4-folds-clr

https://www.kaggle.com/gmhost/gru-capsule

https://github.com/dennybritz/cnn-text-classification-tf

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv', 'embeddings', 'embeddings.zip']


# load package

In [2]:
import os
import time
import random
import re
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


# global parameters

In [3]:
data_dir = "../input/"
train_file = os.path.join(data_dir, "train.csv")
test_file = os.path.join(data_dir, "test.csv")
embedding_size = 300
max_len = 50
max_features = 120000
batch_size = 512

# Data preprocess

In [4]:
# 将特殊字符单独挑出
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
#             x = x.replace(punct, f' {punct} ') # 这是python3.6语法
            x = x.replace(punct, ' '+punct+' ')
    return x

# 清洗数字
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

# 清洗拼写
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [5]:
def load_and_prec():
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    # 小写
    train_df["question_text"] = train_df["question_text"].str.lower()
    test_df["question_text"] = test_df["question_text"].str.lower()
    
    # 数字清洗
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))
    
    # 清洗拼写
    train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))
    
    # 数据清洗
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    # 这个方法把所有字母都小写了
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)
    
    ## Get the target values
    train_Y = train_df['target'].values
    print(np.sum(train_Y))
    
#     # 在pad之前把前30个词去掉
#     train_cut = []
#     test_cut = []
#     for x in train_X:
#         train_cut.append([i for i in x if i>30])
#     for x in test_X:
#         test_cut.append([i for i in x if i>30])
#     train_X = train_cut
#     test_X = test_cut

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=max_len, padding="post", truncating="post")
    test_X = pad_sequences(test_X, maxlen=max_len, padding="post", truncating="post")
    
# #     # 把最常用的40个词去掉，pad为0
# #     train_X = np.where(train_X>=40, train_X, 0)
# #     test_X = np.where(test_X>=40, test_X, 0)
    
    #shuffling the data
    np.random.seed(20190101)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_Y = train_Y[trn_idx]
    
    print(train_X.shape)
    print(train_Y.shape)
    print(test_X.shape)
    print(len(tokenizer.word_index))
    
    return train_X, test_X, train_Y, tokenizer.word_index

# load_and_prec()

# load embeddings

In [6]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

def load_fasttext(word_index):
    """
    这个加载词向量还没有细看
    """
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100 and o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

# Models

text_cnn

In [7]:
class model_text_cnn(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, embedding_matrix, sequence_length=60, num_classes=1,
                 embedding_size=300, filter_sizes=[1,3,5], num_filters=256, trainable=True):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        # The structure of the model
        self.layers(num_classes, filter_sizes, num_filters)
        # optimizer
        if trainable:
#             self.learning_rate = tf.get_variable(
#                     "learning_rate", shape=[], dtype=tf.float32, trainable=False)
            self.learning_rate = tf.train.exponential_decay(
                learning_rate=0.001, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
    
    def layers(self, num_classes, filter_sizes, num_filters):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        with tf.variable_scope("convolutional_layers"):
            """
            卷积和池化操作
            """
            self.pooled_outputs = []
            for i, filter_size in enumerate(filter_sizes):
                with tf.variable_scope("conv-maxpool-{}".format(filter_size)):
                    # conv
                    filter_W = tf.get_variable(shape=[filter_size, embedding_size, num_filters], name="filter_W", dtype=tf.float32)
                    filter_b = tf.get_variable(shape=[num_filters], name="filter_b", dtype=tf.float32)
                    conv = tf.nn.conv1d(
                        value=self.embedding_inputs,
                        filters=filter_W,
                        stride=1,
                        padding="VALID",
                        name="conv") # shape:[batch_size, seq_len-filter_size+1, num_filters]
                    # relu
                    h = tf.nn.relu(tf.nn.bias_add(conv, filter_b), name="relu")
                    # global max pooling layer
                    pooled = tf.reduce_max(h, axis=1, name='global_max_pooling') # shape: [batch_size, num_filters]
                    self.pooled_outputs.append(pooled)
        
            # Combine all the pooled features
            self.conv_outputs = tf.concat(self.pooled_outputs, axis=1)
            # dropout
            self.conv_outputs_drop = tf.nn.dropout(self.conv_outputs, self.keep_prob)
            
#         with tf.variable_scope("fully_connected"):
#             """
#             全连接层
#             """
#             fc_W = tf.get_variable(
#                     shape=[self.conv_outputs_drop.get_shape().as_list()[1], num_classes],
#                     initializer=tf.contrib.layers.xavier_initializer(),
#                     name="fc_w")
#             fc_b = tf.get_variable(shape=[num_classes], initializer=tf.constant_initializer(0.1), name="fc_b")
#             self.logits = tf.nn.bias_add(tf.matmul(self.conv_outputs_drop, fc_W), fc_b)
            
#         with tf.variable_scope("softmax_and_loss"):
#             """
#             softmax输出层并计算loss
#             """
#             self.softmax = tf.nn.softmax(self.logits)
#             self.classes = tf.cast(tf.argmax(self.softmax, axis=1), dtype=tf.int32, name="classes")
#             self.loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
#                     logits=self.logits, labels=self.input_y))
        
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc_W1 = tf.get_variable(
                    shape=[self.conv_outputs_drop.get_shape().as_list()[1], 128],
                    initializer=tf.contrib.layers.xavier_initializer(),
                    name="fc_w1")
            fc_b1 = tf.get_variable(shape=[128], initializer=tf.constant_initializer(0.1), name="fc_b1")
            fc_1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(self.conv_outputs_drop, fc_W1), fc_b1))
            fc_1_drop = tf.nn.dropout(fc_1, self.keep_prob)
            
            fc_W2 = tf.get_variable(
                    shape=[128, num_classes],
                    initializer=tf.contrib.layers.xavier_initializer(),
                    name="fc_w2")
            fc_b2 = tf.get_variable(shape=[num_classes], initializer=tf.constant_initializer(0.1), name="fc_b2")
            self.logits = tf.squeeze(tf.nn.bias_add(tf.matmul(fc_1_drop, fc_W2), fc_b2), name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.classes = tf.cast(tf.greater(self.sigmoid, 0.34), dtype=tf.int32, name="classes")
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
 

# Training Tools

In [8]:
# batch生成器
def batch_generator(train_X, train_Y, batch_size, is_train=True):
    """
    batch生成器:
    在is_train为true的情况下，补充batch，并shuffle
    """
    data_number = train_X.shape[0]
    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size > data_number:
            # 最后一个batch的操作
            if is_train:
                # 后面的直接舍弃，重新开始
                # shuffle
                np.random.seed(2018)
                trn_idx = np.random.permutation(data_number)
                train_X = train_X[trn_idx]
                train_Y = train_Y[trn_idx]
                one_batch_X = train_X[0:batch_size]
                one_batch_Y = train_Y[0:batch_size]
                batch_count = 1
                yield one_batch_X, one_batch_Y
            else:
                one_batch_X = train_X[batch_count * batch_size:data_number]
                one_batch_Y = train_Y[batch_count * batch_size:data_number]
                batch_count = 0
                yield one_batch_X, one_batch_Y
        else:
            one_batch_X = train_X[batch_count * batch_size:batch_count * batch_size + batch_size]
            one_batch_Y = train_Y[batch_count * batch_size:batch_count * batch_size + batch_size]
            batch_count += 1
            yield one_batch_X, one_batch_Y


In [9]:
def f1_smart(y_true, y_pred):
    args = np.argsort(y_pred)
    tp = y_true.sum()
    fs = (tp - np.cumsum(y_true[args[:-1]])) / np.arange(y_true.shape[0] + tp - 1, tp, -1)
    res_idx = np.argmax(fs)
    return 2 * fs[res_idx], (y_pred[args[res_idx]] + y_pred[args[res_idx + 1]]) / 2

In [10]:
# 正类欠采样，负类数据增强，暂时用随机打乱数据增强.
def data_augmentation(X, Y, under_sample=100000, aug_num=3):
    """
    under_sample: 欠采样个数
    aug: 数据增强倍数
    """
    pos_X = []
    neg_X = []
    for i in range(X.shape[0]):
        if Y[i] == 1:
            neg_X.append(list(X[i]))
        else:
            pos_X.append(list(X[i]))
    
    # 正样本欠采样
    random.shuffle(pos_X)
    pos_X = pos_X[:-under_sample]
    
    # 正样本数据增强
    pos_X_aug = []
    for i in range(200000):
        aug = []
        for x in pos_X[i]:
            if x != 0:
                aug.append(x)
            else:
                break
        random.shuffle(aug)
        aug += [0] * (max_len-len(aug))
        pos_X_aug.append(aug)
    pos_X.extend(pos_X_aug)
    print(len(pos_X))
    
    # 负样本数据增强
    neg_X_aug = []
    for i in range(aug_num):
        for neg in neg_X:
            aug = []
            for x in neg:
                if x != 0:
                    aug.append(x)
                else:
                    break
            random.shuffle(aug)
            aug += [0] * (max_len-len(aug))
            neg_X_aug.append(aug)
        
    neg_X.extend(neg_X_aug)
    print(len(neg_X))
    
    pos_Y = np.zeros(shape=[len(pos_X)], dtype=np.int32)
    neg_Y = np.ones(shape=[len(neg_X)], dtype=np.int32)
    
    pos_X.extend(neg_X)
    X_out = np.array(pos_X, dtype=np.int32)
    Y_out = np.append(pos_Y, neg_Y)
    
    print(X_out.shape)
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(X_out))

    X_out = X_out[trn_idx]
    Y_out = Y_out[trn_idx]
    
    print(X_out.shape)
    print(Y_out.shape)

    return X_out, Y_out

# Main part

In [11]:
# 加载数据，平均词向量
train_X, test_X, train_Y, word_index = load_and_prec()
# embedding_matrix_1 = load_glove(word_index)
embedding_matrix = load_fasttext(word_index)
# embedding_matrix_3 = load_para(word_index)
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2], axis = 0)
np.shape(embedding_matrix)
# embedding_matrix = np.zeros(shape=[100,300],dtype=np.float32)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)
80810
(1306122, 50)
(1306122,)
(56370, 50)
185946


(120000, 300)

In [12]:
# 多折训练，交叉验证平均，测试

# 划分交叉验证集
DATA_SPLIT_SEED = 20190101
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_X, train_Y))

# 最终输出
best_test = []

# 多折训练
for i, (train_idx, valid_idx) in enumerate(splits):
    X_train = train_X[train_idx]
    Y_train = train_Y[train_idx]
    X_val = train_X[valid_idx]
    Y_val = train_Y[valid_idx]
    
#     # 数据增强
#     X_train, Y_train = data_augmentation(X_train, Y_train) 
#     print(Y_train[:100])
#     print(Y_train[-100:])
    
    # batch生成器
    train_batch = batch_generator(X_train, Y_train, batch_size, True)
    val_batch = batch_generator(X_val, Y_val, batch_size, False)
    test_batch = batch_generator(test_X, np.zeros(shape=[test_X.shape[0]], dtype=np.int32), batch_size, False)
    
    # 选择最好的结果
    best_val_f1 = 0.0
    
    # 训练 & 验证 & 测试
    with tf.Graph().as_default():
        sess_config = tf.ConfigProto(allow_soft_placement=True)
        sess_config.gpu_options.allow_growth = True
        with tf.Session(config=sess_config) as sess:
            writer = tf.summary.FileWriter("./log/", sess.graph)
            # 模型
            model = model_text_cnn(embedding_matrix=embedding_matrix, sequence_length=max_len)
            sess.run(tf.global_variables_initializer())
#             sess.run(tf.assign(model.learning_rate, tf.constant(0.001, dtype=tf.float32)))
            
            train_loss_sum = 0.0
            for go in range(20000):
                steps = sess.run(model.global_step) + 1
                # 训练
                train_batch_X, train_batch_Y = next(train_batch)
                feed = {model.input_x:train_batch_X, model.input_y:train_batch_Y, model.keep_prob:0.7}
                loss, train_op = sess.run([model.loss, model.train_op], feed_dict=feed)
                train_loss_sum += loss
                # 验证 & 测试
                if steps % 1000 == 0: 
                    val_predictions = []
                    val_loss_sum = 0.0
                    for _ in range(X_val.shape[0] // batch_size + 1):
                        val_batch_X, val_batch_Y = next(val_batch)
                        feed_val = {model.input_x:val_batch_X, model.input_y:val_batch_Y, model.keep_prob:1.0}
                        val_loss, val_classes = sess.run([model.loss, model.classes], feed_dict=feed_val)
                        val_predictions.extend(val_classes)
                        val_loss_sum += val_loss
                    val_f1 = metrics.f1_score(Y_val, np.array(val_predictions))
#                     val_f1, _ = f1_smart(Y_val, np.array(val_predictions))
                    val_pre = metrics.precision_score(Y_val, np.array(val_predictions))
                    val_recall = metrics.recall_score(Y_val, np.array(val_predictions))
                    val_loss_sum = val_loss_sum / (X_val.shape[0] // batch_size + 1)
                    print("steps:{}, train_loss:{:.5f}, val_loss:{:.5f}, val_F1:{:.5f}, val_pre:{:.5f}, val_recall:{:.5f}".format(
                        steps, float(train_loss_sum / 1000), float(val_loss_sum), float(val_f1), float(val_pre), float(val_recall)))
                    
                    # 写入tensorboard
                    train_loss_write = tf.Summary(value=[tf.Summary.Value(tag="model/train_loss", \
                                                                          simple_value=train_loss_sum / 1000), ])
                    writer.add_summary(train_loss_write, steps)
                    val_loss_write = tf.Summary(value=[tf.Summary.Value(tag="model/val_loss", simple_value=val_loss_sum), ])
                    writer.add_summary(val_loss_write, steps)
                    val_f1_write = tf.Summary(value=[tf.Summary.Value(tag="index/val_f1", simple_value=val_f1), ])
                    writer.add_summary(val_f1_write, steps)
                    val_pre_write = tf.Summary(value=[tf.Summary.Value(tag="index/val_precision", simple_value=val_pre), ])
                    writer.add_summary(val_pre_write, steps)
                    val_recall_write = tf.Summary(value=[tf.Summary.Value(tag="index/val_recall", simple_value=val_recall), ])
                    writer.add_summary(val_recall_write, steps)
                    writer.flush()
                    
                    # train loss
                    train_loss_sum = 0.0
                    
                    # 测试，并选取最好的F1值的时刻的测试结果为最终结果
                    if val_f1 > best_val_f1:
                        best_val_f1 = val_f1
                        best_test = []
                        for _ in range(test_X.shape[0] // batch_size + 1):
                            test_batch_X, _ = next(test_batch)
                            feed_test = {model.input_x:test_batch_X, model.keep_prob:1.0}
                            test_classes = sess.run(model.classes, feed_dict=feed_test)
                            best_test.extend(test_classes)
                        print("test done!")
    break # 先只用一折测试

sub = pd.read_csv('../input/sample_submission.csv')
sub["prediction"] = best_test
sub.to_csv("submission.csv", index=False)


steps:1000, train_loss:0.13604, val_loss:0.12437, val_F1:0.60239, val_pre:0.48056, val_recall:0.80695
test done!
steps:2000, train_loss:0.11866, val_loss:0.10965, val_F1:0.63923, val_pre:0.55719, val_recall:0.74960
test done!
steps:3000, train_loss:0.11334, val_loss:0.10757, val_F1:0.64843, val_pre:0.56125, val_recall:0.76766
test done!
steps:4000, train_loss:0.11117, val_loss:0.10566, val_F1:0.65297, val_pre:0.57283, val_recall:0.75919
test done!
steps:5000, train_loss:0.10759, val_loss:0.10408, val_F1:0.66258, val_pre:0.59003, val_recall:0.75548
test done!
steps:6000, train_loss:0.10741, val_loss:0.10472, val_F1:0.66013, val_pre:0.57255, val_recall:0.77936
steps:7000, train_loss:0.10366, val_loss:0.10318, val_F1:0.66507, val_pre:0.59479, val_recall:0.75418
test done!
steps:8000, train_loss:0.10365, val_loss:0.10343, val_F1:0.66119, val_pre:0.56751, val_recall:0.79192
steps:9000, train_loss:0.10008, val_loss:0.10170, val_F1:0.66969, val_pre:0.59114, val_recall:0.77231
test done!
steps